# Week 15 Programming Assignment

The final thing for this semester that we haven't exercised, yet, is working with databases through Python.  In thi final assignment of the semester, you will practice pulling data from either Google Big Query or Snowflake, loading that data into a Pandas data frame, summarizing the data, and then exporting that to an Excel file.

**You will need to use your own Google Big Query or Snowflake account to run this notebook, but you should try to make it configurable so anyone with an account in those technologies could run your notebook with minimal changes.**


Build a notebook with good comments (either in code or in markdown cells).  Then submit your assignment as usual.

### 1. Connect to the data source

Follow the examples provided in the [week15 folder of our GitHub repository](https://github.com/paulboal/hds5210-2023/tree/main/week15) to connect your notebook either to Big Query or Snowflake.

In [1]:
# Installing the packages required
!pip install "snowflake-connector-python[pandas]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.1.0
    Uninstalling platformdirs-4.1.0:
      Successfully uninstalled platformdirs-4.1.0


In [7]:
import snowflake.connector
# my snowflake details
details = {"snowflake_username" : 'VISHAL',
           "snowflake_password" : 'Vishal@2710',
           "snowflake_account" : 'DBAINHO-PL25312',
           "snowflake_warehouse" : 'COMPUTE_WH',
           "snowflake_database" : 'ONCOLOGY_DEMO',
           "snowflake_schema" : 'OMOP',
           "snowflake_query" : 'SELECT * FROM COST;'
           }
           # My SQL Query


### 2. Query some data

Assuming the source database has some data in it or that you can load some data into it from any source, query it.  Then, read that data into a Pandas dataframe.

In [9]:
with snowflake.connector.connect(
        user=details["snowflake_username"],
        password=details["snowflake_password"],
        account=details["snowflake_account"],
        warehouse=details["snowflake_warehouse"],
        database=details["snowflake_database"],
        schema=details["snowflake_schema"]
) as con:

  # Creating a cursor object
  cursor = con.cursor()

  # Executing the above query
  cursor.execute(details["snowflake_query"])

  # This code will fetch all the results into a Pandas DataFrame
  data = cursor.fetch_pandas_all()
  data

### 3. Aggregate your data frame

Do some kind of aggregation on your data frame.  Something that makes sense and has some groups to it.  Don't just sum up one column for the entire data frame.  Be more creative than that.

In [13]:
data

,COST_DATE,TOTAL_CHARGE,TOTAL_PAID
0,2016-09-09,230.0,NaN
1,2016-05-13,2.0,NaN
2,2016-09-23,12.0,NaN
3,2020-04-20,NaN,60.0
4,2016-09-09,NaN,12.0
...,...,...,...
773529,2018-04-13,NaN,6.0
773530,2018-04-13,NaN,0.0
773531,2018-04-13,0.0,NaN
773532,2018-04-13,27.0,NaN


In [19]:
data = data[['COST_DATE','TOTAL_CHARGE','TOTAL_PAID']]
data

,COST_DATE,TOTAL_CHARGE,TOTAL_PAID
0,2016-09-09,230.0,NaN
1,2016-05-13,2.0,NaN
2,2016-09-23,12.0,NaN
3,2020-04-20,NaN,60.0
4,2016-09-09,NaN,12.0
...,...,...,...
773529,2018-04-13,NaN,6.0
773530,2018-04-13,NaN,0.0
773531,2018-04-13,0.0,NaN
773532,2018-04-13,27.0,NaN


In [20]:
data.dtypes

COST_DATE       datetime64[ns]
TOTAL_CHARGE           float64
TOTAL_PAID             float64
dtype: object

In [21]:
import pandas as pd
data['COST_DATE'] = pd.to_datetime(data['COST_DATE'])
data.dtypes

COST_DATE       datetime64[ns]
TOTAL_CHARGE           float64
TOTAL_PAID             float64
dtype: object

In [23]:
data = data.groupby(by = 'COST_DATE').agg('sum').sort_index().reset_index()

### 4. Write to Excel

Use Pandas functions to write your summarized data out to a local Excel file.

In [24]:
data.to_excel('ONCO.xlsx')

---

## Submitting Your Work

Submit your work as usual